In [1]:
# start coding here
token = snakemake.params

NameError: name 'snakemake' is not defined

# PitViper Notebook Report

This notebook was generated automatically by PitViper.

It can be used in two ways:

1. By using the functions already created and present in the following cells.

2. By creating new cells and writing python3 code in them.

The graphs are generated using the python library [Altair](https://altair-viz.github.io/index.html). It is possible to download each graph in SVG format from the drop-down menu at the top right of each graph.

The next cell allows to call the functions already created for the visualization of the results.

In [ ]:
import sys
import os

modules_path = ['workflow/notebooks/', "../../../workflow/notebooks/"]
for module in modules_path:
    module_path = os.path.abspath(os.path.join(module))
    if module_path not in sys.path:
        sys.path.append(module_path)

from functions import * 

working_directory_update(snakemake.output[0])

### rpy2

Next cell load rpy2 which allow to define cells running R code by adding `%%R` at the first line of the cell. 

Note: it's possible to forward some python-specific objects (such as list or pandas dataframe) to R cells using `-i` parameters followed by objects name (`%%R -i token`, token string is then available as an R object).

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

source("workflow/notebooks/functions_R.R")

## Process data

Next function scan `results/` directory to retrieve all results.

`tools_available` is python dictionnary in which all data are stored in a comprehensive manner.

In [ ]:
results_directory, tools_available = setup_step_1(token)

## Mapping Quality Control

If available, mapping quality control metrics will be shown by next function (`show_mapping_qc`).

In [ ]:
show_mapping_qc(token)

## Read count distribution

Normalized read count distribution for all replicates will be shown by calling `show_read_count_distribution` function.

In [ ]:
alt.data_transformers.disable_max_rows()

show_read_count_distribution(token)

## Principal component analysis

In [ ]:
pca_counts(token)

## Tools global results

In [ ]:
snake_plot(results_directory, tools_available)

## sgRNA read counts by element

In [ ]:
show_sgRNA_counts(token) 

In [ ]:
%%R -i token

config_data <- get_data(token)

tsv_file <- config_data$tsv
cts_file <- config_data$cts

mean_counts_by_condition(conditions = "J4,J11,J18,J25",
                         gene = "MYC")

## results by tool and by element

In [ ]:
CRISPhieRmix_results(results_directory, tools_available)

In [ ]:
GSEA_like_results(results_directory, tools_available)

In [ ]:
in_house_method_results(results_directory, tools_available)

In [ ]:
MAGeCK_RRA_results(results_directory, tools_available)

In [ ]:
MAGeCK_MLE_results(results_directory, tools_available)

In [ ]:
BAGEL_results(results_directory, tools_available)

In [ ]:
enrichr_plots(tools_available)

## GeneMania

In [ ]:
genemania_link_results(tools_available)

In [ ]:
intersection(tools_available, token)

## Definition of ranks and occurrences

In next cell, choose which conditions to compare.

In [ ]:
TREATMENT = "J25"
CONTROL = "J4"

In [ ]:
ranks, occurences = ranking(TREATMENT, CONTROL, token, tools_available)

## Venn diagram

In [ ]:
%%R -i occurences,CONTROL,TREATMENT

venn_diagram(occurences, TREATMENT, CONTROL)

## Essentiality prioritisation using RobustRankAggregation

In this section, the `RobustRankAggreg` (RRA) method is used to integrate ranks derived from the results calculated by each selected method.

In [ ]:
%%R -i ranks

res <- RobustRankAggregate(ranks)

res %>% filter(Score < 0.05)

## depmap integration with "Essentiality prioritisation" results

In [ ]:
%%R

library(depmap)
library("ExperimentHub")
library(ggplot2)

## create ExperimentHub query object
eh <- ExperimentHub()

# rnai <- eh[["EH2260"]]

# rnai

In [ ]:
%%R

eh

In [ ]:
# %%R -i

# query.gene <- "MYC"
# query.cell_line <- "KASUMI2_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE"
# query.cell_line_name <- "KASUMI2"

# essential.genes <- res %>%
#     filter(Score < 0.05) %>%
#     pull(Name)


# rnai.filtered <- rnai %>%
#     filter(cell_line == query.cell_line) %>%
#     mutate(essential = ifelse(gene_name %in% essential.genes, "essential", "non-essential")) %>%
#     filter(!is.na(dependency))


# gene.dependency <- rnai.filtered %>%
#     filter(gene_name == query.gene) %>%
#     pull(dependency)


# rnai.filtered %>%
#     ggplot(aes(x=dependency, fill=essential)) + 
#     geom_density(alpha=0.4) + 
#     theme_classic() +
#     scale_fill_manual(values=c("#E69F00", "#999999")) +
#     geom_vline(xintercept = 0) +
#     geom_rug(data = subset(rnai.filtered, essential == "essential"), col="#E69F00", alpha=0.5, sides = "t") +
#     geom_rug(data = subset(rnai.filtered, essential == "non-essential"), col="#999999", alpha=0.5, outside = TRUE, sides = "t") +
#     coord_cartesian(clip = "off") +
#     geom_vline(xintercept = gene.dependency, linetype="dashed", color = "red", size=1) +
#     ggtitle(paste0(query.cell_line_name, "dependencies (", query.gene," highlighted)")) +
#     theme(plot.title = element_text(vjust = 4),
#           plot.margin = margin(10, 10, 10, 10))


In [ ]:
# %%R

# library(ggridges)
# library(tidyr)
# library(forcats)

# rnai.filtered <- rnai %>%
#     filter(grepl("HAEMATOPOIETIC_AND_LYMPHOID_TISSUE", cell_line)) %>%
#     mutate(essential = ifelse(gene_name %in% essential.genes, "essential", "non-essential")) %>%
#     filter(!is.na(dependency))


# rnai.filtered %>%
#     mutate(cell_line = fct_reorder(.f = cell_line, .x = -dependency, .fun = median)) %>%
#     filter(essential == "essential") %>%
#     ggplot(aes(x = `dependency`, y = cell_line, fill = stat(x))) +
#         geom_density_ridges_gradient(scale = 3, rel_min_height = 0.01, quantile_lines = TRUE, quantiles = 2) +
#         scale_fill_viridis_c() +
#         geom_vline(xintercept=-0.5, color = "red", size=1) +
#         geom_vline(xintercept=0, color = "black", size=1, linetype="dashed") +
#         theme_classic()


In [ ]:
# import pandas as pd
# import rpy2.robjects as robjects
# from rpy2.robjects import pandas2ri# Defining the R script and loading the instance in Python
# r = robjects.r
# r['source']('preprocess.R')# Loading the function we have defined in R.
# filter_country_function_r = robjects.globalenv['filter_country']# Reading and processing data
# df = pd.read_csv("Country-Sales.csv")#converting it into r object for passing into r function
# df_r = pandas2ri.ri2py(df)
# #Invoking the R function and getting the result
# df_result_r = filter_country_function_r(df_r, 'USA')
# #Converting it back to a pandas dataframe.
# df_result = pandas2ri.py2ri(df_result_r)

In [ ]:
tools = list(tools_available.keys())

SCHEMES = ["reds", "accent", "redblue", "rainbow"]

def plot_chart(data, tool, condition, file, x, y, method, column_filter='', cutoff=0, greater=False, element='', column_element='', elem_color='', pass_color='', fail_color='', category_column = '', color_scheme='', reverse=''):
    source = data[tool][condition][file].copy()
    show_plot = True
    if method == 'Cut-off':
        if (source[column_filter].dtype in [np.float64, np.int64]):
            if not greater:
                pass_value = "%s < %s" % (column_filter,cutoff)
                fail_value = "%s >= %s" % (column_filter,cutoff)
                source.loc[source[column_filter] < cutoff, 'filter'] = pass_value
                source.loc[source[column_filter] >= cutoff, 'filter'] = fail_value
                source.loc[source[column_element] == element, 'filter'] = element
            else:
                pass_value = "%s >= %s" % (column_filter,cutoff)
                fail_value = "%s < %s" % (column_filter,cutoff)
                source.loc[source[column_filter] >= cutoff, 'filter'] = pass_value
                source.loc[source[column_filter] < cutoff, 'filter'] = fail_value
                source.loc[source[column_element] == element, 'filter'] = element                
            domain = [pass_value, fail_value, element]
            range_ = [pass_color, fail_color, elem_color]
        else:
            source['filter'] = 'pass'
            show_plot = False
            print('Choose a column of integer or float type.')
    if show_plot:
        if method == 'Cut-off':
            chart = alt.Chart(source).transform_calculate(
                order="{'%s': 0, '%s': 1, '%s': 2}[datum.%s]" % (element, pass_value, fail_value, column_element)
            ).mark_circle().encode(
                x=x,
                y=y,
                tooltip = list(source.columns),
                color=alt.Color('filter', scale=alt.Scale(domain=domain, range=range_), legend=alt.Legend(title="Pass filter:")),
                order='order:Q'
            ).interactive()
            display(chart)
        else:
            chart = alt.Chart(source).mark_circle().encode(
                x=x,
                y=y,
                tooltip = list(source.columns),
                color=alt.Color(category_column, legend=alt.Legend(title="%s:" % category_column), scale=alt.Scale(scheme=color_scheme, reverse=reverse))
            ).interactive()
            display(chart)


def call_form():
    display(widgets.HTML(value="First, choose a <b>tool</b> to browse results:"))
    @interact(
        tool = widgets.Dropdown(options=tools,value=tools[0],description='Tool:',disabled=False))
    def form(tool):
        display(widgets.HTML(value="Choose a <b>condition</b> available for this tool:"))
        conditions = list(tools_available[tool].keys())
        @interact(
            condition = widgets.Dropdown(options=conditions,value=conditions[0],description='Condition:',disabled=False))
        def form(condition):
            display(widgets.HTML(value="Choose a <b>file</b>:"))
            files = list(tools_available[tool][condition].keys())
            @interact(file = widgets.Dropdown(options=files,value=files[0],description='File:',disabled=False))
            def form(file):
                display(widgets.HTML(value="Choose which columns to use as <b>x</b> and <b>y</b> axis:"))
                columns = tools_available[tool][condition][file].columns
                @interact(x = widgets.Dropdown(options=columns,value=columns[0],description='x:',disabled=False),
                          y = widgets.Dropdown(options=columns,value=columns[1],description='y:',disabled=False))
                def form(x , y):
                    display(widgets.HTML(value="Results can be colored using two <b>methods</b>.</br> <ul><li>Cut-off: Choose a <b>column</b> with numerical values on which you want to apply a <b>cut-off</b></li><li>Category: choose a column and a color scheme.</li></ul>"))
                    @interact(method = widgets.ToggleButtons(options=['Cut-off', 'Category'],value='Cut-off', description='Method:'))
                    def form(method):
                        if method == 'Cut-off':
                            display(widgets.HTML(value="Now choose a column with numerical data and a set a <b>cut-off</b> value."))
                            @interact(column_filter = widgets.Dropdown(options=columns,value=columns[0],description='Column:',disabled=False),
                                      cutoff = widgets.FloatText(value=0.0,description='Cut-off:'),
                                      greater = widgets.Checkbox(value=False,description='Greater?',disabled=False,indent=True),
                                      pass_color = widgets.ColorPicker(concise=False,description='Pass color:',value='red',disabled=False),
                                      fail_color = widgets.ColorPicker(concise=False,description='Fail color:',value='gray',disabled=False),
                                      column_element = widgets.Dropdown(options=columns,value=columns[0],description='Column:',disabled=False))
                            def form(column_filter, cutoff, greater, pass_color, fail_color, column_element):
                                display(widgets.HTML(value="It is possible to highlight an <b>element</b> based on a column and value from that column."))
                                first_element = str(tools_available[tool][condition][file][column_element][0])
                                @interact(element = widgets.Text(value=first_element,placeholder='Element:',description='Element:',disabled=False),
                                          elem_color = widgets.ColorPicker(concise=False,description='Element color:',value='blue',disabled=False))
                                def form(element, elem_color):
                                    plot_chart(tools_available, tool, condition, file, x, y, method, column_filter, cutoff, greater, element, column_element, elem_color, pass_color, fail_color)
                        elif method == 'Category':
                            @interact(category_column = widgets.Dropdown(options=columns,value=columns[0],description='Category:',disabled=False),
                                      reverse = widgets.Checkbox(value=False,description='Reverse?',disabled=False,indent=True),
                                      color_scheme = widgets.Dropdown(options=SCHEMES,value=SCHEMES[0],description='Scheme:',disabled=False))
                            def form(category_column, reverse, color_scheme):
                                plot_chart(tools_available, tool, condition, file, x, y, method, category_column=category_column, color_scheme=color_scheme, reverse=reverse)

In [ ]:
call_form()

In [ ]:
import numpy as np
import pandas as pd
from clustergrammer2 import net, Network, CGM2
import warnings
# warnings.filterwarnings('ignore')

In [ ]:
df = pd.DataFrame(np.random.randint(0,10,size=(10, 4)), columns=['BRCA2','TP53','SPNS2','MYC'])

print(df)

# load matrix file
net.load_df(df)

# cluster using default parameters
# net.cluster(enrichrgram=True)

# make interactive widget
net.widget()

In [ ]:
from ipywidgets import Button, HBox, VBox

TOOLS = [tool for tool in tools_available.keys() if tool != "DESeq2"]

# Define widgets's options
conditions_options = tools_available[TOOLS[0]].keys()
tools_options = TOOLS

# Define widgets
conditions_widget = widgets.Dropdown(options=conditions_options)
tools_widget = widgets.SelectMultiple(options=tools_options)

params = {'MAGeCK_MLE': {'on': False,
                         'fdr': 0.05,
                         'score': 0,
                         'greater': True},
          'MAGeCK_RRA': {'on': False,
                         'fdr': 0.05,
                         'score': 0,
                         'greater': True},
          'BAGEL': {'on': False,
                         'score': 0,
                         'greater': True},
          'CRISPhieRmix': {'on': False,
                         'fdr': 0.05,
                         'score': 0,
                         'greater': True},
          'in_house_method': {'on': False,
                         'direction': None,
                         'score': 0,
                         'greater': True},
          'GSEA_like': {'on': False,
                         'fdr': 0.05,
                         'score': 0,
                         'greater': True}}


# Show form selection
def show_tools(condition_selected, tools_selected):
    
    def mle_order_update(change):
        with output:
            if change['new'] == 'Greater than score':
                params['MAGeCK_MLE']['greater'] = True
            else:
                params['MAGeCK_MLE']['greater'] = False
    def mle_fdr_update(change):
        with output:
            params['MAGeCK_MLE']['fdr'] = change['new']
    def mle_score_update(change):
        with output:
            params['MAGeCK_MLE']['score'] = change['new']
            
    def rra_order_update(change):
        with output:
            if change['new'] == 'Greater than score':
                params['MAGeCK_RRA']['greater'] = True
            else:
                params['MAGeCK_RRA']['greater'] = False
    def rra_fdr_update(change):
        with output:
            params['MAGeCK_RRA']['fdr'] = change['new']
    def rra_score_update(change):
        with output:
            params['MAGeCK_RRA']['score'] = change['new']

    def bagel_order_update(change):
        with output:
            if change['new'] == 'Greater than score':
                params['BAGEL']['greater'] = True
            else:
                params['BAGEL']['greater'] = False
    def bagel_fdr_update(change):
        with output:
            params['BAGEL']['fdr'] = change['new']
    def bagel_score_update(change):
        with output:
            params['BAGEL']['score'] = change['new']

    def CRISPhieRmix_order_update(change):
        with output:
            if change['new'] == 'Greater than score':
                params['CRISPhieRmix']['greater'] = True
            else:
                params['CRISPhieRmix']['greater'] = False
    def CRISPhieRmix_fdr_update(change):
        with output:
            params['CRISPhieRmix']['fdr'] = change['new']
    def CRISPhieRmix_score_update(change):
        with output:
            params['CRISPhieRmix']['score'] = change['new']
            
    def in_house_method_order_update(change):
        with output:
            if change['new'] == 'Greater than score':
                params['in_house_method']['greater'] = True
            else:
                params['in_house_method']['greater'] = False
    def in_house_method_direction_update(change):
        with output:
            params['in_house_method']['direction'] = change['new']
    def in_house_method_score_update(change):
        with output:
            params['in_house_method']['score'] = change['new']
            
            
    def GSEA_like_order_update(change):
        with output:
            if change['new'] == 'Greater than score':
                params['GSEA_like']['greater'] = True
            else:
                params['GSEA_like']['greater'] = False
    def GSEA_like_fdr_update(change):
        with output:
            params['GSEA_like']['fdr'] = change['new']
    def GSEA_like_score_update(change):
        with output:
            params['GSEA_like']['score'] = change['new']
            
    def on_button_clicked(b):
        with output:
            print(params)
            print(condition_selected)
            treatment, control = condition_selected.split("_vs_")
            ranking(treatment, control, token, tools_available, params)
            
    # Output
    output = widgets.Output()
    button = widgets.Button(description="Click Me!")
    if 'MAGeCK_MLE' in tools_selected:
        params['MAGeCK_MLE']['on'] = True
        mle_fdr = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='FDR:')
        mle_score=widgets.FloatText(value=0, description='Score cut-off:')
        mle_text=widgets.HTML(value="MAGeCK MLE:")
        mle_order=widgets.ToggleButtons(options=['Greater than score', 'Lower than score'], description='Selection:', name="test")
        display(mle_text)
        mle_box = HBox([mle_fdr, mle_score, mle_order])
        display(mle_box)
        mle_order.observe(mle_order_update, 'value')
        mle_score.observe(mle_score_update, 'value')
        mle_fdr.observe(mle_fdr_update, 'value')
    if 'MAGeCK_RRA' in tools_selected:
        params['MAGeCK_RRA']['on'] = True
        rra_fdr = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='FDR:')
        rra_score=widgets.FloatText(value=0, description='Score cut-off:')
        rra_text=widgets.HTML(value="MAGeCK RRA:")
        rra_order=widgets.ToggleButtons(options=['Greater than score', 'Lower than score'], description='Selection:')
        display(rra_text)
        rra_box = HBox([rra_fdr, rra_score, rra_order])
        display(rra_box)
        rra_order.observe(rra_order_update, 'value')
        rra_score.observe(rra_score_update, 'value')
        rra_fdr.observe(rra_fdr_update, 'value')
    if 'BAGEL' in tools_selected:
        params['BAGEL']['on'] = True
        bagel_score=widgets.FloatText(value=0, description='Score cut-off:')
        bagel_text=widgets.HTML(value="BAGEL:")
        bagel_order=widgets.ToggleButtons(options=['Greater than score', 'Lower than score'], description='Selection:')
        display(bagel_text)
        bagel_box = HBox([bagel_score, bagel_order])
        display(bagel_box)
        bagel_order.observe(bagel_order_update, 'value')
        bagel_score.observe(bagel_score_update, 'value')
    if 'CRISPhieRmix' in tools_selected:
        params['CRISPhieRmix']['on'] = True
        CRISPhieRmix_fdr = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='FDR:')
        CRISPhieRmix_score=widgets.FloatText(value=0, description='Score cut-off:')
        CRISPhieRmix_text=widgets.HTML(value="CRISPhieRmix:")
        CRISPhieRmix_order=widgets.ToggleButtons(options=['Greater than score', 'Lower than score'], description='Selection:')
        display(CRISPhieRmix_text)
        CRISPhieRmix_box = HBox([CRISPhieRmix_fdr, CRISPhieRmix_score, CRISPhieRmix_order])
        display(CRISPhieRmix_box)
        CRISPhieRmix_order.observe(CRISPhieRmix_order_update, 'value')
        CRISPhieRmix_score.observe(CRISPhieRmix_score_update, 'value')
        CRISPhieRmix_fdr.observe(CRISPhieRmix_fdr_update, 'value')
    if 'in_house_method' in tools_selected:
        params['in_house_method']['on'] = True
        in_house_method_direction=widgets.ToggleButtons(options=['Positive', 'Negative'], description='Direction:')
        in_house_method_score=widgets.FloatText(value=0, description='Score cut-off:')
        in_house_method_text=widgets.HTML(value="in_house_method:")
        in_house_method_order=widgets.ToggleButtons(options=['Greater than score', 'Lower than score'], description='Selection:')
        display(in_house_method_text)
        in_house_method_box = HBox([in_house_method_direction, in_house_method_score, in_house_method_order])
        display(in_house_method_box)
        in_house_method_order.observe(in_house_method_order_update, 'value')
        in_house_method_score.observe(in_house_method_score_update, 'value')
        in_house_method_direction.observe(in_house_method_direction_update, 'value')
    if 'GSEA-like' in tools_selected:
        params['GSEA_like']['on'] = True
        GSEA_like_fdr = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='FDR:')
        GSEA_like_score=widgets.FloatText(value=0, description='Score cut-off:')
        GSEA_like_text=widgets.HTML(value="GSEA-like:")
        GSEA_like_order=widgets.ToggleButtons(options=['Greater than score', 'Lower than score'], description='Selection:')
        display(GSEA_like_text)
        GSEA_like_box = HBox([GSEA_like_fdr, GSEA_like_score, GSEA_like_order])
        display(GSEA_like_box)
        GSEA_like_order.observe(GSEA_like_order_update, 'value')
        GSEA_like_score.observe(GSEA_like_score_update, 'value')
        GSEA_like_fdr.observe(GSEA_like_fdr_update, 'value')
    display(button, output)
    button.on_click(on_button_clicked)
    
        
_ = interact(show_tools, condition_selected=conditions_widget, tools_selected=tools_widget)